In [5]:
print(300)
print(300, 300)
for _ in range(1,150):
    print(1,_)
    print(_,1)

300
300 300
1 1
1 1
1 2
2 1
1 3
3 1
1 4
4 1
1 5
5 1
1 6
6 1
1 7
7 1
1 8
8 1
1 9
9 1
1 10
10 1
1 11
11 1
1 12
12 1
1 13
13 1
1 14
14 1
1 15
15 1
1 16
16 1
1 17
17 1
1 18
18 1
1 19
19 1
1 20
20 1
1 21
21 1
1 22
22 1
1 23
23 1
1 24
24 1
1 25
25 1
1 26
26 1
1 27
27 1
1 28
28 1
1 29
29 1
1 30
30 1
1 31
31 1
1 32
32 1
1 33
33 1
1 34
34 1
1 35
35 1
1 36
36 1
1 37
37 1
1 38
38 1
1 39
39 1
1 40
40 1
1 41
41 1
1 42
42 1
1 43
43 1
1 44
44 1
1 45
45 1
1 46
46 1
1 47
47 1
1 48
48 1
1 49
49 1
1 50
50 1
1 51
51 1
1 52
52 1
1 53
53 1
1 54
54 1
1 55
55 1
1 56
56 1
1 57
57 1
1 58
58 1
1 59
59 1
1 60
60 1
1 61
61 1
1 62
62 1
1 63
63 1
1 64
64 1
1 65
65 1
1 66
66 1
1 67
67 1
1 68
68 1
1 69
69 1
1 70
70 1
1 71
71 1
1 72
72 1
1 73
73 1
1 74
74 1
1 75
75 1
1 76
76 1
1 77
77 1
1 78
78 1
1 79
79 1
1 80
80 1
1 81
81 1
1 82
82 1
1 83
83 1
1 84
84 1
1 85
85 1
1 86
86 1
1 87
87 1
1 88
88 1
1 89
89 1
1 90
90 1
1 91
91 1
1 92
92 1
1 93
93 1
1 94
94 1
1 95
95 1
1 96
96 1
1 97
97 1
1 98
98 1
1 99
99 1
1 100
100 1
1 10